PieceValues(): 이 함수는 체스 말의 가치를 정의
BlankChess(values): 이 함수는 빈 체스 보드를 생성합니다

InitChess(values): 이 함수는 초기화된 체스 보드를 생성합니다

PrintBoard(board): 이 함수는 현재 체스 보드를 출력합니다.

Move(board, oldloc, newloc): 이 함수는 말을 이동시킵니다. 이동 전 위치와 이동 후 위치를 매개변수로 받아 이동을 수행하고, 이동된 말의 이름과 가치를 반환합니다.

PawnMove(board, i, j): 폰의 가능한 이동 위치를 계산합니다.

GoFar(board, i, j, poss, kk, ll): 이 함수는 특정 방향으로 원하는 위치까지 이동 가능한 경우를 찾dk


ScoreActivePieces(board): 각 플레이어의 활성 말 세아림

AllPossible(board, i, j): 특정 위치에 있는 말의 가능한 모든 이동 위치를 계산

  #ScoreCenterControl(board): 보드의 중앙을 통제하는 정도에 따라 플레이어의 점수를 계산

  #ScorePeril(board): 위협 받고 있는 말의 가치에 따라 플레이어의 점수를 계산

Score(board, player): 플레이어의 점수를 계산
    

  #ScoreMoves(board, player): 가능한 모든 이동에 대한 점수를 계산합니다.
PickMove(moves): 가능한 이동 중 가장 좋은 이동을 선택
SortMoves(moves): 이동을 점수순으로 정렬

PickMoveFrom5(moves): 가능한 이동 중 5개의 이동을 선택

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 19 11:22:43 2020

@author: jkinser
"""
# (C) Jason Kinser 2022.
# This code is intended for non-commercial use in courses at George Mason Univ.
# Their are no guarantees with this code
# Commercial and/or non-academic use prohibited without written permission from the author.


# ##########  CHESS

import copy
import numpy as np
#%%
def PieceValues():
    values = {}
    values['p'] = 100#pawn
    values['r'] = 500#rook 십자모양
    values['n'] = 325#night#3단계 L자모양으로만
    values['b'] = 325#bishop#x자모양
    values['q'] = 1050#queen#night 모양 빼고 다
    values['k'] = 40000#king#한칸만
    return values   
#%%
def BlankChess(values):
    blank = [' .',0]
    board = np.zeros((8,8),type(blank))
    for i in range(8):
        for j in range(8):
            board[i,j] = blank
    return board
#%%
def InitChess(values):
    blank = [' .',0]
    board = np.zeros((8,8),type(blank))
    for i in range(8):
        for j in range(8):
            board[i,j] = blank
    board[0,0] = ['br1',values['r']] # creates a rook.  black rook #1
    board[0,1] = ['bn1',values['n']] # knight
    board[0,2] = ['bb1',values['b']] # bishop
    board[0,3] = ['bq',values['q']] # queen
    board[0,4] = ['bk',values['k']] # king
    board[0,5] = ['bb2',values['b']]
    board[0,6] = ['bn2',values['n']]
    board[0,7] = ['br2',values['r']]
    for i in range(8):
        board[1,i] = ['bp'+str(i+1),1]
    board[7,0] = ['wr1',values['r']] # creates a rook.  black rook #1
    board[7,1] = ['wn1',values['n']] # knight
    board[7,2] = ['wb1',values['b']] # bishop
    board[7,3] = ['wq',values['q']] # queen
    board[7,4] = ['wk',values['k']] # king
    board[7,5] = ['wb2',values['b']]
    board[7,6] = ['wn2',values['n']]
    board[7,7] = ['wr2',values['r']]
    for i in range(8):
        board[6,i] = ['wp'+str(i+1),1]
    return board
#%%
def PrintBoard(board):
    for v in range(8):
        for h in range(8):
            print('%5s' % board[v,h][0], end='')
        print('')
#%%
# assume the move is legal
def Move(board, oldloc, newloc):
    # old location and new location
    nam,val = ' .',0
    if board[oldloc][0] != ' .':
        nam, val = board[newloc] # get the name of the piece
        board[newloc] = board[oldloc] # move piece
        board[oldloc] = [' .',0]
    return nam,val # piece that was removed
#%%
def PawnMove(board, i,j ):
    poss = [] # possible moves
    bw = board[i][j][0][0] # black or white
    if bw == 'b':
        # piece is coming down the board
####################꼭 처음에 두 스페이스 움직일 필요 없을듯
        # move one space if unoccupied
        if board[i+1][j][0] == ' .':
            poss.append((i+1,j))
        # move two spaces if unoccupied and at row 1
        if i==1 and board[i+2][j][0]== ' .' and board[i+1][j][0]==' .':
            poss.append((i+2,j))
        # capture one space
        if j-1>=0:
            if board[i+1][j-1][0][0]=='w':
                poss.append((i+1,j-1))
        if j+1<8:
            if board[i+1][j+1][0][0]=='w':
                poss.append((i+1,j+1))
    # repeat for white
    if bw=='w':
        if board[i-1][j][0]==' .':
            poss.append((i-1,j))
        if i==6 and board[i-2][j][0]==' .' and board[i-1][j][0]==' .':
            poss.append((i-2,j))
        if j+1<8:
            if board[i-1][j+1][0][0]=='b':
                poss.append((i-1,j+1))
        if j-1>=0:
            if board[i-1][j-1][0][0]=='b':
                poss.append((i-1,j-1))
    return poss
#%%
def GoFar(board,i,j,poss,kk,ll):
    bw = board[i][j][0][0]
    # move up left
    ok = True
    k = i;l=j
    while ok:
        k += kk; l+= ll
        if k<0 or l<0 or k>7 or l>7:
            ok = False
            break # you have reached the end of the board        
        if board[k][l][0]==' .':
            poss.append((k,l))  # open space
        if (board[k][l][0][0]=='w' and bw=='b') or (board[k][l][0][0]=='b' and bw=='w'):
            poss.append((k,l)) # atrack
            ok = False
            break
        if board[k][l][0][0]== bw:
            ok = False
            break
    # don't need to return poss
    
def RookMove(board,i,j):
    poss = []
    GoFar(board,i,j,poss,-1, 0)
    GoFar(board,i,j,poss, 1, 0)
    GoFar(board,i,j,poss, 0,-1)
    GoFar(board,i,j,poss, 0, 1)
    return poss

    
def BishopMove(board, i,j):
    poss = []
    GoFar(board,i,j,poss,-1,-1)
    GoFar(board,i,j,poss,-1, 1)
    GoFar(board,i,j,poss, 1,-1)
    GoFar(board,i,j,poss, 1, 1)
    return poss

def QueenMove(board,i,j):
    poss = RookMove(board,i,j)
    poss.extend( BishopMove(board,i,j))
    return poss
#%%
def KnightMove(board,i,j):
    bw = board[i][j][0][0]
    poss = []
    allowed = ((1,2),(-1,2),(1,-2),(-1,-2),(2,1),(-2,1),(-2,-1),(2,-1))
    for kk,ll in allowed:
        k = i+kk; l=j+ll
        if k<0 or k>7 or l<0 or l>7:
            pass # off the board
        elif board[k][l][0][0]==bw:
            # landing on your on piece
            pass
        else:
            poss.append((k,l)) 
    return poss
#%%
def KingMove(board,i,j):
    bw = board[i][j][0][0]
    poss = []
    for kk in range(-1,2):
        for ll in range(-1,2):
            k = i+kk; l=j+ll
            if k<0 or k>7 or l<0 or l>7:
                pass # off the board
            elif board[k][l][0][0]==bw:
                # landing on your on piece
                pass
            else:
                poss.append((k,l)) 
    return poss
#%%
def ScoreActivePieces(board):
    white = 0; black = 0
    # go through each tile
    for i in range(8):
        for j in range(8):
            bw = board[i][j][0][0] # b or w
            if bw=='w':
                white += board[i][j][1]
            if bw=='b':
                black += board[i][j][1]
    return black,white
#%%
def AllPossible(board, i,j):
    # decode piece and determine which function to use
    p = board[i][j][0][1]
    if p=='p':
        poss = PawnMove(board,i,j)
    if p=='r':
        poss = RookMove(board,i,j)
    if p=='n':
        poss = KnightMove(board,i,j)
    if p=='b':
        poss = BishopMove(board,i,j)
    if p=='q':
        poss = QueenMove(board,i,j)
    if p=='k':
        poss = KingMove(board,i,j)
    return poss
#%%
# score center control.
# score pieces in 2x2 middle
# consider all pieces and all their possible moves.  Score these.
def ScoreCenterControl(board):
    black, white = 0,0
    for i in range(3,5):
        for j in range(3,5):
            bw = board[i][j][0][0] # b or w
            if bw=='w':
                white += board[i][j][1]
            if bw=='b':
                black += board[i][j][1]
    # find the pieces that can get to the center of the board.
    for i in range(8):
        for j in range(8):
            poss=[]  # init because a piece may not be able to move
            # if there is a piece, find all possible moves
            if board[i][j][0] != ' .':
                poss = AllPossible(board,i,j)
            # pare down to just the center four and score
            for ii,jj in poss:
                if 3<=ii<=4 and 3<=jj<=4:
                    bw = board[i][j][0][0]
                    if bw=='w':
                        white += board[i][j][1]
                    if bw=='b':
                        black += board[i][j][1]
    return black, white#각각의 값
#%%
# score how many points are in immediate peril
def ScorePeril(board):#먹을 수 있는 점수를 메기고 가장 높은 점수 애를 체택
    black, white = 0,0
    # go through all pieces
    for i in range(8):
        for j in range(8):
            # is there a piece at this position?
            if board[i][j][0]!=' .':
                # get possible moves for this piece
                poss = AllPossible(board,i,j)
                # for each move determine if it captures an opponent
                bw = board[i][j][0][0] # color of current piece
                for px,py in poss:
                    pbw = board[px][py][0][0] # color on target square
                    if pbw != bw and pbw != ' ':
                        # score capture
                        if bw=='b':
                            black += board[px][py][1]
                        if bw=='w':
                            white += board[px][py][1]
    return black, white###만약 capture할 수 있는 말이 두 개면? 그리고 여기서 white는 말의 갯수인가 아님 그 말의 value인가?
####scores for capturing each of those pieces will be added together.
#%%
def Score(board, player='b'):
    # default is black player.  for white use player=0
    # return score as a percentage B/(B+W)
    b1,w1 = ScoreActivePieces(board)
    b2,w2 = ScoreCenterControl(board)
    b3,w3 = ScorePeril(board)
    #print('Scores', b1,w1, b2,w2, b3,w3)
    # more points possible for Active than other two methods.  Scale by 5
    b = b1+5*b2+5*b3; w =w1+5*w2+5*w3
    r = np.random.rand()*0.01 # to break ties
    if player=='b':
        pct = b/(b+w+r)
    else:
        pct = w/(b+w+r)
    return pct
#%%
# score the possible moves
def ScoreMoves(board, player='b'):
    moves = []
    for i in range(8):
        for j in range(8):
            if board[i][j][0][0] == player:
                # this is a black piece
                poss = AllPossible(board,i,j) # all moves for this piece
                for ii,jj in poss:
                    tempboard = copy.deepcopy(board) # reset board
                    Move(tempboard,(i,j),(ii,jj))
                    score = Score(tempboard,player) # score this 
                    moves.append((board[i][j][0],(i,j),(ii,jj),score))
    return moves
#%%
def PickMove(moves):
    best = []; bestsc = 0
    for m in moves:
        if m[3]>bestsc:
            bestsc = m[3]
            best = copy.deepcopy(m)
    # second best move 0 just for grins
    best2, best2sc = [],0
    for m in moves:
        if m[3]>best2sc and m[3]<bestsc:
            best2sc = m[3]
            best2 = copy.deepcopy(m)
    return best, best2
#%%
def SortMoves(moves):
    NMoves = len(moves)
    scores = np.zeros( NMoves )
    for i in range( NMoves ):
        scores[i] = moves[i][3] # collect scores
    ndx = scores.argsort()
    ndx = ndx[::-1]
    answ = []
    for i in ndx:
        answ.append( moves[i] )
    return answ

#%%
def PickMoveFrom5(moves):
    smoves = SortMoves( moves )
    p = [0,1,2,3,4]
    me = np.random.choice(p)
    return smoves[me]


In [2]:
def SelectedMove(board, color):
    """
    Choose the best move based on the current board position and team color.

    Parameters:
    - board (numpy.ndarray): The current state of the chess board.
    - color (str): The color of the player making the move, either 'w' for white or 'b' for black.

    Returns:
    - tuple: A tuple containing the selected move, represented as (old_location, new_location).
    """

    # Check if the current player's king is in check
    king_location = None
    for i in range(8):
        for j in range(8):
            if board[i][j][0] == color + 'k':
                king_location = (i, j)
                break
        if king_location:
            break

    if king_location:
        in_check = IsInCheck(board, king_location, color)
    else:
        in_check = False

    # Get all possible moves for the given color
    all_possible_moves = []
    for i in range(8):
        for j in range(8):
            if board[i][j][0][0] == color:
                # Get possible moves for this piece
                poss_moves = AllPossible(board, i, j)
                # Add each possible move to the list
                for new_location in poss_moves:
                    all_possible_moves.append(((i, j), new_location))

    # Score the possible moves
    moves_with_scores = ScoreMoves(board, color)

    # If in check, prioritize moves that get the king out of check
    if in_check:
        # Find moves that get the king out of check
        safe_moves = []
        for old_location, new_location in all_possible_moves:
            temp_board = copy.deepcopy(board)
            Move(temp_board, old_location, new_location)
            new_king_location = new_location if board[old_location[0]][old_location[1]][0] == color + 'k' else king_location
            if not IsInCheck(temp_board, new_king_location, color):
                safe_moves.append((old_location, new_location))

        # If there are safe moves, prioritize them
        if safe_moves:
            return safe_moves[0][0], safe_moves[0][1]

    # Select the best move from the list of possible moves
    best_move = PickMoveFrom5(moves_with_scores)

    return best_move[1:]  # Return only the old and new locations of the selected move


In [3]:
def FindKing(board, player):
    for row_index, row in enumerate(board):
        for col_index, piece in enumerate(row):
            if piece[0] == player + 'k':  # 'wk' for white king, 'bk' for black king
                return row_index, col_index

def IsCheck(board, current_player):
    # Find the position of the current player's king
    king_position = FindKing(board, current_player)
    
    # Iterate through all opponent's pieces to see if any can attack the king
    opponent = 'b' if current_player == 'w' else 'w'
    for piece_position in FindPieces(board, opponent):
        if IsValidMove(board, piece_position, king_position):
            return True
    
    # If no opponent's piece can attack the king, it's not in check
    return False

def Checkmate(board, current_player):
    # Check if the current player's king is in check
    if IsCheck(board, current_player):
        # Iterate through all possible moves for the current player
        for move in GenerateAllPossibleMoves(board, current_player):
            # Make the move temporarily
            temp_board = copy.deepcopy(board)
            Move(temp_board, move[1], move[2])
            # If after the move, the king is no longer in check, return False
            if not IsCheck(temp_board, current_player):
                return False
        # If no move can get the king out of check, it's checkmate
        return True
    else:
        # If the king is not in check, it's not checkmate
        return False


In [4]:
def FindPieces(board, player):
    pieces = []
    for row_index, row in enumerate(board):
        for col_index, cell in enumerate(row):
            if cell[0].startswith(player):
                pieces.append((row_index, col_index))
    return pieces

In [13]:

values = PieceValues()
board = InitChess(values)  # You can also use BlankChess(values) for an empty board

def IsScholarsMatePossible(board, color):
    if color == 'w':
        # Check if the board is in the initial state and the necessary moves are possible
        if board[1, 4] == 'wp' and board[6, 4] == 'bp':
            return True
        if board[0, 3] == 'wq' and board[7, 3] == 'bq':
            return True
        if board[0, 5] == 'wb' and board[7, 5] == 'bb':
            return True
    return False

def ScholarsMateMove(board, color, move_count):
    if color == 'w' and move_count == 0:
        Move(board, (1, 4), (3, 4))  # e2 to e4
        return True
    if color == 'w' and move_count == 1 and board[3, 4] == 'wp':
        Move(board, (0, 5), (4, 1))  # Bc4
        return True
    if color == 'w' and move_count == 2 and board[4, 1] == 'wb':
        Move(board, (0, 3), (4, 7))  # Qh5
        return True
    if color == 'w' and move_count == 3 and board[4, 7] == 'wq':
        Move(board, (4, 7), (6, 5))  # Qf7#
        return True
    return False

# Main game loop
current_player = 'w'  # Start with white
move_count = 0

while True:
    # Print the current board state
    print('')
    PrintBoard(board)

    # Check for Scholar’s Mate
    if move_count < 4 and IsScholarsMatePossible(board, current_player): 
        if ScholarsMateMove(board, current_player, move_count):
            print(f"Scholar's Mate executed by {current_player}")
            move_count += 1
            continue
    
    # Get legal moves for the current player
    legal_moves = ScoreMoves(board, current_player)
    
    # Select the best move
    best_move, _ = PickMove(legal_moves)  # Or use PickMoveFrom5 for a more random choice
    
    # Make the selected move
    old_location = best_move[1]
    new_location = best_move[2]
    Move(board, old_location, new_location)
    
    # Check for game-ending conditions
    king_position = FindKing(board, current_player)
    if king_position is None:
        print(f"Checkmate!")
        break
    
    # Switch to the other player
    current_player = 'b' if current_player == 'w' else 'w'
    move_count += 1



  br1  bn1  bb1   bq   bk  bb2  bn2  br2
  bp1  bp2  bp3  bp4  bp5  bp6  bp7  bp8
    .    .    .    .    .    .    .    .
    .    .    .    .    .    .    .    .
    .    .    .    .    .    .    .    .
    .    .    .    .    .    .    .    .
  wp1  wp2  wp3  wp4  wp5  wp6  wp7  wp8
  wr1  wn1  wb1   wq   wk  wb2  wn2  wr2

  br1  bn1  bb1   bq   bk  bb2  bn2  br2
  bp1  bp2  bp3  bp4  bp5  bp6  bp7  bp8
    .    .    .    .    .    .    .    .
    .    .    .    .    .    .    .    .
    .    .    .    .    .    .    .    .
    .    .  wn1    .    .    .    .    .
  wp1  wp2  wp3  wp4  wp5  wp6  wp7  wp8
  wr1    .  wb1   wq   wk  wb2  wn2  wr2

  br1  bn1  bb1   bq   bk  bb2    .  br2
  bp1  bp2  bp3  bp4  bp5  bp6  bp7  bp8
    .    .    .    .    .  bn2    .    .
    .    .    .    .    .    .    .    .
    .    .    .    .    .    .    .    .
    .    .  wn1    .    .    .    .    .
  wp1  wp2  wp3  wp4  wp5  wp6  wp7  wp8
  wr1    .  wb1   wq   wk  wb2  wn2  wr2

  br1  bn1  